# Hybrid Monte Carlo

## Payoff Scripting

In this notebook we demonstrate the setup and use of *Payoff* objects. This is structured along the following steps:

  1.  Specifying and using basic payoffs
  2.  Combining basic payoffs to form complex payoff structures
  3.  Simulate future payoffs with Monte Carlo
  4.  Set up payoffs for American Monta Carlo

We use a couple of standard packages for calculation and analysis

In [ ]:
import sys
sys.path.append('../')  # make python find our modules
import numpy as np
import pandas as pd
import plotly.express as px

The following auxilliary method lists the relevant members of an object. We use it to inspect the objects created. 

In [ ]:
def members(obj):
    return [f for f in dir(obj) if not f.startswith('_')] 

Initially, we want to focus on payoff specification and leave modelling and simulation to a later stage. However, to be able to *use* a payoff we need a proxy model. Such proxy model can be a *DeterministicModel*.

The *DeterministicModel* provides a *Path* which we can use for proxy payoff calculation.

In [ ]:
from src.termstructures.YieldCurve import YieldCurve
from src.models.DeterministicModel import DeterministicModel
model = DeterministicModel('EUR', YieldCurve(0.03),
    forAliases=['USD', 'SX5E'], forAssetSpots=[0.85, 3200.0],
    forCurves=[YieldCurve(0.02),YieldCurve(0.01)])
p = model.path()

### Specifying and using basic payoffs

A payoff represents a function mapping a *path* to an observable scalar output. The specific function is encoded in the *Payoff* objects.

A *Payoff* has an observation time. This is the time the payoff is finally known or paid (depending on context). For a payoff we can calculate future simulated values $V(t)$ for a path via *at(path)* method.

Moreover, we can calculate future (simulated) discounted values via *discountedAt(path)* method. This method calculates $V(t)/B(t)$.



In [ ]:
details = lambda o : 't: %.2f, at: %.4f, discountedAt: %.4f' % \
    (o.obsTime,o.at(p),o.discountedAt(p))

We list some main payoffs and explain their meaning.

In [ ]:
from src.simulations.Payoffs import Fixed, Asset, LiborRate, Pay

The most basic payoff is a deterministic amount, here *Fixed(.)*

In [ ]:
a = Fixed(3.14159)
display(details(a))

The amount is known immediately. Numeraire at zero is one. Consequently, discounted value equals future value (in this example).

Future FX rates, single stock or index values are represented via *Asset()* payoffs.

In [ ]:
fx = Asset(10.0,'USD')
display(details(fx))
#
s = Asset(5.0,'SX5E')
display(details(s))


Now observation time is different from zero. Thus discounted value differs from un-discounted simulated value.

One of the most import rates payoffs is a *LiborRate()* payoff. As the name suggests it calculates a future simple compounde rate as
$$
  L(t,T_s,T_e)= \left[ \frac{P(t,T_s)}{P(t,T_e)}D - 1 \right]
                \frac{1}{\tau}.
$$
Discount factors $P(t,T)$ are calculated from the model. Tenor basis factor $D$ and year fraction should be provided by user.

In [ ]:
L = LiborRate(10.0,10.0,10.5,alias='EUR')  # here 6m Libor rate
display(details(L))

Often, payoffs are paid at a time later than their observation time. For exaple, a Libor rate is fixed at time $t$ (equal to or shortly before start time $T_s$). But a floating rate cash flow is only paid at the end of the interest rate period at $T_e$.

This is important for Monte Carlo simulation because for derivative pricing we need to discount a cash flow from its pay time. The pay time specifies the time the numeraire $B(t)$ is evaluated at.

Pay times for payoffs are specified via *Pay()* payoff. This is a payoff that decorates an existing payoff with a new observation time for numeraire calculation.

In [ ]:
C = Pay(L,10.5)
display(details(C))

We see that the simulated value (*at*) remains unchanged. But the discounted value changes slightly due to the changed pay time.

### Combining basic payoffs to form complex payoff structures

We implement several arithmetic functions and operators that can be used with *Payoff* objects to compose more complex cash flow structures.

We start with an example for a Vanilla option on Euro Stoxx 50.

In [ ]:
from src.simulations.Payoffs import Max
T_ex   = 10.0
T_pay  = 10.0 + 7/365.0  # one week payment delay
strike = 3000.0
cop    = -1.0  # call (+1) or put (-1) option
#
V      = Max(cop*(Asset(T_ex,'SX5E')-strike),0.0) @ T_pay

Now that payoffs become more complex it is important to have means to inspect and double check them. For this purpose we implement string conversion for payoff objects.

In [ ]:
display(str(V))

Note that we use the *@* operator to abbreviate the *Pay()* payoff.

Again we can check the simulated value for our payoff given our proxy model.

In [ ]:
display(details(V))

Since we specify a put options and strike is below forward the simulated option value is zero.

As a second example we consider a forward Libor cash flow in foreign currency. Such a cash flow will be a building block for exposure calculation of cross currency swaps.

In [ ]:
from src.simulations.Payoffs import ZeroBond
T  = 5.0  # observation AND pay date of discounted payoff

Nt = 1000.0                              # notional
L  = LiborRate(T,10.0,10.5,alias='USD')  # forward Libor since T<<T_s
S  = 0.01                                # a spread of 1% added on top of Libor rate
YF = 0.5                                 # year fraction for coupon
DF = ZeroBond(T,10.5,'USD')              # discount from T_e to T
FX = Asset(T,'USD')                      # convert USD to EUR
#
V  = (FX * Nt * (L+S) * YF * DF) @ T


The composed payoff can be displayed again.

In [ ]:
display(str(V))

We can simplify the payoff slightly by re-ordering factors and let python calculate known values.


In [ ]:
V  = (Nt * YF * FX * (L+S) * DF) @ T
display(str(V))

The simulated value in domestic currency (here EUR) can be calculated as before.

In [ ]:
display(details(V))

This all works well as long as we use payoffs valid for a given model. But what happens if specify a different payoff?

In [ ]:
S = Asset(5.0,'DAX')
display(str(S))
try:
    display(details(S))
except Exception as e:
    display(e)


We see, we can set up the payoff and view it. But we can not use it with our model because we did not specify dynamics for asset *DAX*.

### Simulate future payoffs with Monte Carlo

So far we only used a proxy discounted cash flow model. Such a proxy model is well suited to test and debug payoffs. However, for actual modelling we want to use a stochastic model and a corresponding simuation.

We re-use our complex model.

In [ ]:
import pickle
with open('model.dump','rb') as f:
    model = pickle.load(f)
display(model)

And simulate it as before. Now, we use considerably more paths for a more realistic simulation.

In [ ]:
from src.simulations.MCSimulation import MCSimulation
seed = 314159265359
nPaths = 2**10
times = np.linspace(0.0, 10.0, 11)
mcsim = MCSimulation(model,times,nPaths,seed)

We can ask the simulation for a given path and use that path to value a payoff.

In [ ]:
p = mcsim.path(0)  # we pick the first path as an example
display(details(V))

With Monte Carlo method we calculate present values by estimating the expectation
$$
  V(0) = \mathbb{E}\left[ \frac{V(t)}{B(t)} \right]
         \approx \frac{1}{N} \sum_{i=1}^N \frac{V^i(t)}{B^i(t)}.
$$
Here, $N$ is the number of Monte Carlo paths, $V^i$ are simulated future payoffs and $B^i$ are simulated future numeraires.

The fraction $V^i(t)/B^i(t)$ is returned by *discountedAt()* for a given input path $i$.

The present value for a payoff can now easily be calculated.

In [ ]:
display(str(V))
V_samples = np.array([ V.discountedAt(mcsim.path(i))
    for i in range(mcsim.nPaths) ])
display('Av: %.4f, stdErr: %.4f' % \
    (np.average(V_samples),np.std(V_samples)/np.sqrt(mcsim.nPaths)))
#
df = pd.DataFrame(V_samples,columns=['$V^i(0)$'])
fig = px.histogram(df,x=df.columns[0])
fig.show()

We have a look at another example, a Vanilla put on Eurostoxx 50.

In [ ]:
V = Max(-1*(Asset(10.0,'SX5E')-3000.0),0.0) @ 10.0
V_samples = np.array([ V.discountedAt(mcsim.path(i))
    for i in range(mcsim.nPaths) ])
display('Av: %.4f, stdErr: %.4f' % \
    (np.average(V_samples),np.std(V_samples)/np.sqrt(mcsim.nPaths)))
#
df = pd.DataFrame(V_samples,columns=['$V^i(0)$'])
fig = px.histogram(df,x=df.columns[0])
fig.show()

### Set up payoffs for American Monte Carlo

Future conditional expectations of complex payoffs may be approximated by American Monte Carlo (AMC) methods. These methods are typically based on regression. They allow approximating
$$
  V(t) = B(t) \cdot \mathbb{E}\left[ \frac{V(T)}{B(T)} \, | \, \cal{F}_t  \right].
$$

We implement AMC by linear regression using control variables. Calibration of the regression requires another Monte Carlo simulation. This is an input to the AMC payoff.

In [ ]:
mcsim_training = MCSimulation(model,times,nPaths,2718281828)

To illustrate the methodology we give the example of the Euro Stoxx 50 put option observed in 5y. As control variables we consider $S_{SX5E}(5.0)$ and $[S_{SX5E}(5.0)-3000]^+$.

In [ ]:
from src.simulations.AmcPayoffs import AmcSum
C0 = Asset(5.0,'SX5E')
C1 = Max(C0-3000.0,0.0)
maxDegree = 2  # the maximum polynomial degree for linear regression
V1 = AmcSum(5.0,[V],[C0,C1],mcsim_training,maxDegree)
display(str(V1))


We will explore applications for AMC payoffs in the context of exposure simulation.

For this example we just calculate the present value of the AMC payoff.

In [ ]:
V_samples = np.array([ V1.discountedAt(mcsim.path(i))
    for i in range(mcsim.nPaths) ])
display('Av: %.4f, stdErr: %.4f' % \
    (np.average(V_samples),np.std(V_samples)/np.sqrt(mcsim.nPaths)))
#
df = pd.DataFrame(V_samples,columns=['$V_1^i(0)$'])
fig = px.histogram(df,x=df.columns[0])
fig.show()

Because of Tower Law we have $V_1(0)\approx V(0)$ (up to errors by the numerical scheme). However, the variance (and thus stdErr) of $V_1$ is considerably smaller than the variance of $V$. 